<a href="https://colab.research.google.com/github/Masud-Sagar/Skill-Morph-Data-Science/blob/main/Assignment_5_Data_Preprocessing_and_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
#Load the Data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [ ]:
# TODO: Load the housing data
df = pd.read_csv('/content/drive/MyDrive/dataset/Housing.csv')

In [ ]:
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [11]:
# TODO: How many houses are there?
#print(f"Number of houses: ___")  # Fill this

# TODO: Show first 3 houses
# Your code here
print(f"Number of houses: ", df.count().sum())
df.head(3)

Number of houses:  7085


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished


In [18]:
# Look at Columns
# TODO: Show all column names
#print("Columns:")
# Your code here
print("Columns:", df.columns.tolist())

# TODO: Show data types of columns
print("Data types:", df.dtypes)

Columns: ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea', 'furnishingstatus']
Data types: price                int64
area                 int64
bedrooms             int64
bathrooms            int64
stories              int64
mainroad            object
guestroom           object
basement            object
hotwaterheating     object
airconditioning     object
parking              int64
prefarea            object
furnishingstatus    object
dtype: object


In [44]:
#Convert Yes/No to 1/0
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['mainroad_n'] = label_encoder.fit_transform(df['mainroad'])
df['guestroom_n'] = label_encoder.fit_transform(df['guestroom'])
df['basement_n'] = label_encoder.fit_transform(df['basement'])
df['hotwaterheating_n'] = label_encoder.fit_transform(df['hotwaterheating'])
df['airconditioning_n'] = label_encoder.fit_transform(df['airconditioning'])
df['prefarea_n'] = label_encoder.fit_transform(df['prefarea'])

furniture_encoded = pd.get_dummies(df['furnishingstatus'], prefix='furnishingstatus')
print("\nOne-Hot Encoding for 'furnishingstatus':")
print(furniture_encoded)
# Creates separate columns for each city

# Add to original data
df = pd.concat([df, furniture_encoded], axis=1)
print("\nFinal data with all encodings:")
print(df)

# Check what's in our data
print("Current mainroad values:", df['mainroad'].head())
# Shows: yes, no, yes, yes, no

# Models need numbers, not text!
print("Current furnishingstatus_n values:", df['furnishingstatus_n'].head())
# We need: yes → 1, no → 0


One-Hot Encoding for 'furnishingstatus':
     furnishingstatus_furnished  furnishingstatus_semi-furnished  \
0                          True                            False   
1                          True                            False   
2                         False                             True   
3                          True                            False   
4                          True                            False   
..                          ...                              ...   
540                       False                            False   
541                       False                             True   
542                       False                            False   
543                        True                            False   
544                       False                            False   

     furnishingstatus_unfurnished  
0                           False  
1                           False  
2                           False

In [45]:
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,...,mainroad_n,guestroom_n,basement_n,hotwaterheating_n,airconditioning_n,prefarea_n,furnishingstatus_n,furnishingstatus_furnished,furnishingstatus_semi-furnished,furnishingstatus_unfurnished
0,13300000,7420,4,2,3,yes,no,no,no,yes,...,1,0,0,0,1,1,0,True,False,False
1,12250000,8960,4,4,4,yes,no,no,no,yes,...,1,0,0,0,1,0,0,True,False,False
2,12250000,9960,3,2,2,yes,no,yes,no,no,...,1,0,1,0,0,1,1,False,True,False
3,12215000,7500,4,2,2,yes,no,yes,no,yes,...,1,0,1,0,1,1,0,True,False,False
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,...,1,1,1,0,1,0,0,True,False,False


In [50]:
# Split into Train and Test
df = df.drop(['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus'], axis=1)

X = df.drop('price', axis=1)
y = df['price']

print("X has all information except price")
print("y has price (1 column)")
# TODO: Split - 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,    # 20% for test
    random_state=20   # Same result every time
)

print(f"\nWe will train with: {len(X_train)} household")
print(f"We will test with: {len(X_test)} household")

X has all information except price
y has price (1 column)

We will train with: 436 household
We will test with: 109 household


In [62]:
df.head()

,price,area,bedrooms,bathrooms,stories,parking,mainroad_n,guestroom_n,basement_n,hotwaterheating_n,airconditioning_n,prefarea_n,furnishingstatus_n,furnishingstatus_furnished,furnishingstatus_semi-furnished,furnishingstatus_unfurnished
0,13300000,7420,4,2,3,2,1,0,0,0,1,1,0,True,False,False
1,12250000,8960,4,4,4,3,1,0,0,0,1,0,0,True,False,False
2,12250000,9960,3,2,2,2,1,0,1,0,0,1,1,False,True,False
3,12215000,7500,4,2,2,3,1,0,1,0,1,1,0,True,False,False
4,11410000,7420,4,1,2,2,1,1,1,0,1,0,0,True,False,False


In [55]:
#Apply StandardScaler
# Area is huge (1650-13300), Bedrooms is small (1-6)!
#Your Code here
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nAfter scaling:")
print("All features now have similar range!")
print("X_train_scaled:")
print(X_train_scaled)
print("X_test_scaled:")
print(X_test_scaled)


After scaling:
All features now have similar range!
X_train_scaled:
[[-0.40096913 -1.33392104 -0.56274914 ... -0.57735027  1.15934724
  -0.69135093]
 [-0.30022929 -1.33392104 -0.56274914 ... -0.57735027  1.15934724
  -0.69135093]
 [-0.54292071 -2.71209539 -0.56274914 ... -0.57735027 -0.86255435
   1.44644342]
 ...
 [-1.48850146  2.800602   -0.56274914 ... -0.57735027  1.15934724
  -0.69135093]
 [-0.36799973  1.42242765 -0.56274914 ... -0.57735027 -0.86255435
   1.44644342]
 [ 1.41692702  0.0442533  -0.56274914 ...  1.73205081 -0.86255435
  -0.69135093]]
X_test_scaled:
[[ 2.87765465  1.42242765 -0.56274914 ... -0.57735027  1.15934724
  -0.69135093]
 [-1.16567607  1.42242765  3.36298887 ... -0.57735027  1.15934724
  -0.69135093]
 [ 0.66137824  1.42242765  1.40011987 ... -0.57735027 -0.86255435
   1.44644342]
 ...
 [-0.30022929  1.42242765  1.40011987 ...  1.73205081 -0.86255435
  -0.69135093]
 [-0.07127511  0.0442533  -0.56274914 ...  1.73205081 -0.86255435
  -0.69135093]
 [-1.07867348 

In [58]:
#train and test model
#your code here
from sklearn.linear_model import LinearRegression

# Create model
model = LinearRegression()

# Train the model
model.fit(X_train_scaled, y_train)
print("✅ Model trained!")

# Check accuracy on training data
train_score = model.score(X_train_scaled, y_train)
print(f"Training accuracy: {train_score:.1%}")

# Check accuracy on test data
test_score = model.score(X_test_scaled, y_test)
print(f"Test accuracy: {test_score:.1%}")

✅ Model trained!
Training accuracy: 67.8%
Test accuracy: 67.6%


In [60]:
#Cross Validation
from sklearn.model_selection import cross_val_score

# Do 5-fold cross-validation
scores = cross_val_score(
    model,           # Our model
    X_train_scaled,  # Training data
    y_train,         # Training labels
    cv=5            # 5 mini-tests
)

print("5 mini-test scores:")
for i in range(5):
    print(f"  Test {i+1}: {scores[i]:.1%}")

print(f"\nAverage: {scores.mean():.1%}")
print(f"This means our model is {scores.mean():.1%} accurate!")

5 mini-test scores:
  Test 1: 66.1%
  Test 2: 71.1%
  Test 3: 62.4%
  Test 4: 63.0%
  Test 5: 62.6%

Average: 65.0%
This means our model is 65.0% accurate!


In [61]:
import joblib

# Save the model
joblib.dump(model, 'my_model.pkl')
print("✅ Model saved as 'my_model.pkl'")

# Save the scaler
joblib.dump(scaler, 'my_scaler.pkl')
print("✅ Scaler saved as 'my_scaler.pkl'")

# Save scaled data as CSV
pd.DataFrame(X_train_scaled).to_csv('train_data_scaled.csv', index=False)
print("✅ Scaled data saved as 'train_data_scaled.csv'")

✅ Model saved as 'my_model.pkl'
✅ Scaler saved as 'my_scaler.pkl'
✅ Scaled data saved as 'train_data_scaled.csv'


In [71]:
# Predict a House Price
# New house details:
# area=5000, bedrooms=3, bathrooms=2, stories=2,
# mainroad=yes, guestroom=no, basement=yes,
# hotwaterheating=no, airconditioning=yes,
# parking=2, prefarea=yes, furnishingstatus=furnished
#your code here
model = joblib.load('my_model.pkl')
scaler = joblib.load('my_scaler.pkl')

# New house data (8 features)
print("\nNew house information:")
new_house = [[5000, 3, 2, 2, 1, 0, 1, 0, 1, 2, 1, 1, 1, 0, 0]]
print("area: 5000")
print("Bedroom: 3")
print("stories: 2")
print("mainroad: yes")
print("(and other features...)")

# Scale the new house data
new_house_scaled = scaler.transform(new_house)

# Make prediction
prediction = model.predict(new_house_scaled)

print(f"\nPredicted price: {prediction[0]:.1f}$")


New house information:
area: 5000
Bedroom: 3
stories: 2
mainroad: yes
(and other features...)

Predicted price: 7799827.6$
